In [1]:
# VADER: https://towardsdatascience.com/sentiment-analysis-in-10-minutes-with-rule-based-vader-and-nltk-72067970fb71

In [24]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [17]:
data = pd.read_pickle("./data/cleaned_submissions.pkl")
data = data.loc[data["sentiment"] != "", ["text", "sentiment"]] # Only get labeled instances

In [20]:
# Encode labels
lenc = LabelEncoder()
y = lenc.fit_transform(data["sentiment"])

# Vectorize text using tfidf
tfidf = TfidfVectorizer(preprocessor=' '.join, lowercase=False, min_df=5) # min_df = Minimum occurance of words
X = tfidf.fit_transform(data["text"])

In [25]:
# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

# SVM

In [29]:
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV

pipeline = Pipeline([
        ("svc", SVC())
    ])

param_grid = {"svc__kernel": ["poly"], "svc__C": [3]} #["poly", "rbf", "sigmoid", "linear"], [3, 4, 5, 6, 7]
CV = GridSearchCV(pipeline, param_grid, cv = 5)
# pipeline.get_params().keys() See all available parameters
CV.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=Pipeline(steps=[('svc', SVC())]),
             param_grid={'svc__C': [3], 'svc__kernel': ['poly']})

In [30]:
print("Accuracy of {} can be achieved with the following parameters: {}".format(CV.score(X_test, y_test), CV.best_params_))

Accuracy of 0.6056808688387636 can be achieved with the following parameters: {'svc__C': 3, 'svc__kernel': 'poly'}


# Naive Bayes

In [31]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV

pipeline = Pipeline([
        ("mnb", MultinomialNB())
    ])

param_grid = {"mnb__alpha": [1]}
CV = GridSearchCV(pipeline, param_grid, cv = 5)
# pipeline.get_params().keys() See all available parameters
CV.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=Pipeline(steps=[('mnb', MultinomialNB())]),
             param_grid={'mnb__alpha': [1]})

In [32]:
print("Accuracy of {} can be achieved with the following parameters: {}".format(CV.score(X_test, y_test), CV.best_params_))

Accuracy of 0.5970481759955444 can be achieved with the following parameters: {'mnb__alpha': 1}


# LSTM

In [37]:
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D

In [68]:
print(X_train_lstm.shape[1:])

(14364, 4936, 1)
(14364,)


In [77]:
print(X_train_lstm.shape[:2])

(14364, 4936)


In [70]:
X_train_lstm = X_train.toarray()[:, :, None]
y_train_lstm = y_train.toarray()[:, None]

AttributeError: 'numpy.ndarray' object has no attribute 'toarray'

In [82]:
model = Sequential()
model.add(LSTM(units=6, input_shape = X_train_lstm.shape[1:], return_sequences = True))
model.add(LSTM(units=6, return_sequences=True))
model.add(LSTM(units=6, return_sequences=True))
# model.add(LSTM(units=1, return_sequences=True, name='output'))
model.add(Dense(1, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics = ['accuracy'])

print(model.summary())

model.fit(X_train_lstm, y_train, epochs=1, verbose=1)

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_28 (LSTM)              (None, 4936, 6)           192       
                                                                 
 lstm_29 (LSTM)              (None, 4936, 6)           312       
                                                                 
 lstm_30 (LSTM)              (None, 4936, 6)           312       
                                                                 
 dense_6 (Dense)             (None, 4936, 1)           7         
                                                                 
Total params: 823
Trainable params: 823
Non-trainable params: 0
_________________________________________________________________
None


ValueError: in user code:

    File "c:\DEV\Master Thesis\App\.thesis\lib\site-packages\keras\engine\training.py", line 878, in train_function  *
        return step_function(self, iterator)
    File "c:\DEV\Master Thesis\App\.thesis\lib\site-packages\keras\engine\training.py", line 867, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\DEV\Master Thesis\App\.thesis\lib\site-packages\keras\engine\training.py", line 860, in run_step  **
        outputs = model.train_step(data)
    File "c:\DEV\Master Thesis\App\.thesis\lib\site-packages\keras\engine\training.py", line 809, in train_step
        loss = self.compiled_loss(
    File "c:\DEV\Master Thesis\App\.thesis\lib\site-packages\keras\engine\compile_utils.py", line 201, in __call__
        loss_value = loss_obj(y_t, y_p, sample_weight=sw)
    File "c:\DEV\Master Thesis\App\.thesis\lib\site-packages\keras\losses.py", line 141, in __call__
        losses = call_fn(y_true, y_pred)
 

In [67]:
X_train_lstm.shape

(14364, 4936, 1)

In [65]:
y_train

array([1, 1, 0, ..., 2, 1, 0])